```
--- Day 16: Ticket Translation ---

As you're walking to yet another connecting flight, you realize that one of the legs of your re-routed trip coming up is on a high-speed train. However, the train ticket you were given is in a language you don't understand. You should probably figure out what it says before you get to the train station after the next flight.

Unfortunately, you can't actually read the words on the ticket. You can, however, read the numbers, and so you figure out the fields these tickets must have and the valid ranges for values in those fields.

You collect the rules for ticket fields, the numbers on your ticket, and the numbers on other nearby tickets for the same train service (via the airport security cameras) together into a single document you can reference (your puzzle input).

The rules for ticket fields specify a list of fields that exist somewhere on the ticket and the valid ranges of values for each field. For example, a rule like class: 1-3 or 5-7 means that one of the fields in every ticket is named class and can be any value in the ranges 1-3 or 5-7 (inclusive, such that 3 and 5 are both valid in this field, but 4 is not).

Each ticket is represented by a single line of comma-separated values. The values are the numbers on the ticket in the order they appear; every ticket has the same format. For example, consider this ticket:

.--------------------------------------------------------.
| ????: 101    ?????: 102   ??????????: 103     ???: 104 |
|                                                        |
| ??: 301  ??: 302             ???????: 303      ??????? |
| ??: 401  ??: 402           ???? ????: 403    ????????? |
'--------------------------------------------------------'
Here, ? represents text in a language you don't understand. This ticket might be represented as 101,102,103,104,301,302,303,401,402,403; of course, the actual train tickets you're looking at are much more complicated. In any case, you've extracted just the numbers in such a way that the first number is always the same specific field, the second number is always a different specific field, and so on - you just don't know what each position actually means!

Start by determining which tickets are completely invalid; these are tickets that contain values which aren't valid for any field. Ignore your ticket for now.

For example, suppose you have the following notes:

class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12
It doesn't matter which position corresponds to which field; you can identify invalid nearby tickets by considering only whether tickets contain values that are not valid for any field. In this example, the values on the first nearby ticket are all valid for at least one field. This is not true of the other three nearby tickets: the values 4, 55, and 12 are are not valid for any field. Adding together all of the invalid values produces your ticket scanning error rate: 4 + 55 + 12 = 71.

Consider the validity of the nearby tickets you scanned. What is your ticket scanning error rate?
```

# part1

In [10]:
function getOthers(lines)
    start = false
    others = []
    for l ∈ lines 
        if start
           push!(others,map( x -> parse(Int,x), split(l,",")))
        end
        if startswith(l,"nearby tickets:")
            start = true
        end
    end
    others
end

lines = readlines("sample.txt")
getOthers(lines)

4-element Array{Any,1}:
 [7, 3, 47]
 [40, 4, 50]
 [55, 2, 20]
 [38, 6, 12]

In [70]:
lines = readlines("sample.txt")

function p1(str)
    x = split(str,"-")
    (parse(Int,x[1]),parse(Int,x[2]))
end

function valid(z,(x,y),(r,s))
    z ∈ x:y || z ∈ r:s
end

function getRules(lines)
    R = Dict()
    for l ∈ lines

        if length(l) == 0
           break 
        end
        x =split(l,": ")
        R[x[1]] = p1.(split(x[2]," or "))
    end
    R
end
lines = readlines("sample.txt")
getRules(lines)

Dict{Any,Any} with 3 entries:
  "class" => [(1, 3), (5, 7)]
  "row"   => [(6, 11), (33, 44)]
  "seat"  => [(13, 40), (45, 50)]

In [37]:
valid(7,(1,7))

true

In [56]:
function invalid(R,A)
    result = copy(A)
    for (k,v) ∈ R
        result = filter( x -> !valid(x,v[1],v[2]), result)
    end 
    result
end

pass1 = filter( x -> !valid(x,(1,3),(5,7)) , [40, 4, 50] )
pass2 = filter( x -> !valid(x,(6,11),(33,44)) , pass1 )
pass3 = filter( x -> !valid(x,(13,40),(45,50)), pass2 )

invalid(R,[40, 4, 50])

1-element Array{Int64,1}:
 4

In [72]:
lines = readlines("sample.txt")
R = getRules(lines)
others = getOthers(lines)
tot = [] 
for o ∈ others
   x = invalid(R,o)
   append!(tot,x)
end
sum(tot)

71

In [73]:
lines = readlines("input.txt")
R = getRules(lines)
others = getOthers(lines)
tot = [] 
for o ∈ others
   x = invalid(R,o)
   append!(tot,x)
end
sum(tot)

20058

```
--- Part Two ---

Now that you've identified which tickets contain invalid values, discard those tickets entirely. Use the remaining valid tickets to determine which field is which.

Using the valid ranges for each field, determine what order the fields appear on the tickets. The order is consistent between all tickets: if seat is the third field, it is the third field on every ticket, including your ticket.

For example, suppose you have the following notes:

class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19

your ticket:
11,12,13

nearby tickets:
3,9,18
15,1,5
5,14,9
Based on the nearby tickets in the above example, the first position must be row, the second position must be class, and the third position must be seat; you can conclude that in your ticket, class is 12, row is 11, and seat is 13.

Once you work out which field is which, look for the six fields on your ticket that start with the word departure. What do you get if you multiply those six values together?
```

In [221]:
lines = readlines("sample2.txt")
 
function allvalid(lines)
    R = getRules(lines)
    others = getOthers(lines)
    xs=Array{Int,2}
    for (i,o) ∈ enumerate(others)
       x = invalid(R,o)
       if length(x) == 0
           xs = [xs;o'] 
        end
    end  
    xs[2:end,:]
end
xs = allvalid(lines)

3×3 Array{Any,2}:
  3   9  18
 15   1   5
  5  14   9

In [220]:
println(xs[:,1])

Any[3, 15, 5]


In [254]:
function invalid(R,A)
    result = copy(A)
    for (k,v) ∈ R
        result = filter( x -> !valid(x,v[1],v[2]), result)
    end 
    result
end

pass1 = filter( x -> !valid(x,(1,3),(5,7)) , [40, 4, 50] )
pass2 = filter( x -> !valid(x,(6,11),(33,44)) , pass1 )
pass3 = filter( x -> !valid(x,(13,40),(45,50)), pass2 )

invalid(R,[40, 4, 50])


2-element Array{Int64,1}:
 40
 50

In [261]:
function invalid2(rule, A)
    ys = filter( x -> !valid(x,rule[1],rule[2]), A)
    length(ys) > 0
    
end
invalid2( [(0, 1), (4, 19)], [3, 15, 5])

true

In [369]:
using Combinatorics
lines = readlines("sample2.txt")
R = getRules(lines)
keys(R)
options = allvalid(lines)
for p ∈ permutations(collect(keys(R)))
    allgood = true
    println("trying $(collect(enumerate(p)))")
    for (col,rule) ∈ enumerate(p)
        y = invalid2(R[rule],options[:,col])
        if y 
            allgood=false
        end
    end
    if allgood == true
        println("done")
        println( collect(enumerate(p)))
        break
    end
end



trying Tuple{Int64,SubString{String}}[(1, "class"), (2, "row"), (3, "seat")]
trying Tuple{Int64,SubString{String}}[(1, "class"), (2, "seat"), (3, "row")]
trying Tuple{Int64,SubString{String}}[(1, "row"), (2, "class"), (3, "seat")]
done
Tuple{Int64,SubString{String}}[(1, "row"), (2, "class"), (3, "seat")]


In [370]:
myticket = [139 109 61 149 101 89 103 53 107 59 73 151 71 67 97 113 83 163 137 167]

1×20 Array{Int64,2}:
 139  109  61  149  101  89  103  53  …  71  67  97  113  83  163  137  167

In [374]:
lines = readlines("input.txt")
using Combinatorics
R = getRules(lines)
keys(R)
os = allvalid(lines)
keys(R)

Base.KeySet for a Dict{Any,Any} with 20 entries. Keys:
  "departure date"
  "zone"
  "price"
  "arrival station"
  "wagon"
  "row"
  "duration"
  "departure station"
  "arrival location"
  "train"
  "departure track"
  "departure time"
  "route"
  "departure location"
  "class"
  "departure platform"
  "arrival platform"
  "arrival track"
  "seat"
  "type"

In [377]:
x = []
R2 = Dict()
for k ∈ keys(R)
    x = []
    for i ∈ 1:20
        if !invalid2(R[k],os[:,i])
            push!(x,i)
        end
    end
    R2[k] = x
end
R2 # now find which one only has one possible solution (and removing this option for all others)

Dict{Any,Any} with 20 entries:
  "departure date"     => Any[1, 2, 5, 6, 7, 8, 9, 11, 12, 13, 17, 18, 19, 20]
  "zone"               => Any[2, 7, 11, 12, 13, 17, 18, 20]
  "price"              => Any[2, 7, 9, 11, 12, 13, 17, 18, 20]
  "arrival station"    => Any[2, 12, 17, 18]
  "wagon"              => Any[2, 12, 17, 18, 20]
  "row"                => Any[2, 7, 11, 12, 17, 18, 20]
  "duration"           => Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 17…
  "departure station"  => Any[1, 2, 3, 5, 6, 7, 8, 9, 11, 12, 13, 17, 18, 19, 2…
  "arrival location"   => Any[2, 7, 12, 17, 18, 20]
  "train"              => Any[12, 18]
  "departure track"    => Any[2, 5, 6, 7, 9, 11, 12, 13, 17, 18, 20]
  "departure time"     => Any[1, 2, 5, 6, 7, 9, 11, 12, 13, 17, 18, 20]
  "route"              => Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 17, 18…
  "departure location" => Any[1, 2, 5, 6, 7, 8, 9, 11, 12, 13, 17, 18, 20]
  "class"              => Any[12, 17, 18]
  "departure platform" => 

In [382]:
solved = sum(length.(values(R2)))
while solved != 20
    for k ∈ keys(R2)
        if length(R2[k]) == 1
           value = R2[k][1] # this is only possible allowed value for this rule
           println("$k is $value")
           for k2 ∈ keys(R2)
                if k2 != k
                    R2[k2] = filter(x -> x != value, R2[k2])
                   # must remove from here as can only be used for k         
                end
            end
        end
    end
    solved = sum(length.(values(R2)))
end
R2

Dict{Any,Any} with 20 entries:
  "departure date"     => Any[19]
  "zone"               => Any[13]
  "price"              => Any[9]
  "arrival station"    => Any[2]
  "wagon"              => Any[20]
  "row"                => Any[11]
  "duration"           => Any[15]
  "departure station"  => Any[3]
  "arrival location"   => Any[7]
  "train"              => Any[12]
  "departure track"    => Any[5]
  "departure time"     => Any[1]
  "route"              => Any[4]
  "departure location" => Any[8]
  "class"              => Any[17]
  "departure platform" => Any[10]
  "arrival platform"   => Any[16]
  "arrival track"      => Any[6]
  "seat"               => Any[18]
  "type"               => Any[14]

In [384]:
myticket = [139 109 61 149 101 89 103 53 107 59 73 151 71 67 97 113 83 163 137 167]
sum(length.(values(R2)))
fields = [v[1] for (k,v) ∈ R2 if startswith(k,"departure") ]
prod([ v for (i,v) ∈ enumerate(myticket) if i ∈ fields])

366871907221